$19^{th}$ Aug 2021

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import glob, os, re
import matplotlib.pyplot as plt
from datetime import  datetime
from ipywidgets import widgets, interactive
from collections import Counter

In [2]:
dt = datetime.today().strftime(format='%d-%m-%Y')

In [3]:
latest_data = 'metadata-seq-MS-ILL-ONT_uptoseq21_10-11-2021.xlsx'
sars_dir = 'SARS-CoV-2' #name of root directory for all SARS-associated work
home_dir = os.getenv('HOME') #get OS home directory
parent_dir = glob.glob(f'{home_dir}/**/{sars_dir}', recursive=True)[0]

In [4]:
#load the excel sheet containing the variants information

df_meta_seq = pd.read_excel(glob.glob(f'{parent_dir}/**/{latest_data}', recursive=True)[0], usecols=['S_NUM', 'S', 'DT_SAM_COLL', 'G_COV'])

In [5]:
df_meta_seq.head()

,S_NUM,DT_SAM_COLL,G_COV,S
0,COVC03617,2020-07-06,98.6,D614G
1,COVC03665,2020-07-06,97.5,"A263S, D614G"
2,COVC03696,2020-07-06,98.7,D614G
3,COVC03723,2020-07-06,98.9,D614G
4,COVC03813,2020-07-10,97.9,D614G


In [6]:
df_meta_seq80 = df_meta_seq.replace(pd.NA, np.nan)[df_meta_seq['G_COV'] >= 80]#.sort_values('DT_SAM_COLL')

In [7]:
df_data = df_meta_seq80[df_meta_seq80.index.isin(df_meta_seq80['DT_SAM_COLL'].dropna().index)]

In [8]:
df_data_dt = df_data.assign(DT_SAM_COLL=df_data['DT_SAM_COLL'].apply(lambda x: pd.to_datetime(x, dayfirst=True))).sort_values('DT_SAM_COLL')

In [9]:
df_data_dt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1274 entries, 0 to 1819
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   S_NUM        1274 non-null   object        
 1   DT_SAM_COLL  1274 non-null   datetime64[ns]
 2   G_COV        1274 non-null   float64       
 3   S            1272 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 49.8+ KB


In [10]:
df_data_dt.tail()

,S_NUM,DT_SAM_COLL,G_COV,S
1815,COVC24707,2021-10-06,96.0,"T19R, L452R, T478K, L518L, D614G, P681R, D950N..."
1816,COVC24708,2021-10-06,82.1,"D614G, P681R,"
1817,COVC24710,2021-10-06,95.9,"T19R, E156_R158delinsG, L452R, T478K, D614G, P..."
1818,COVC24718,2021-10-08,97.0,"T19R, E156_R158delinsG, L452R, T478K, D614G, P..."
1819,COVC24719,2021-10-08,92.6,"T19R, L452R, T478K, D614G, P681R, D950N,"


	S_NUM	DT_SAM_COLL	G_COV	S
1066	COVC24214	2021-08-23	96.7	T19R, L452R, T478K, D614G, P681R, D950N,
1067	COVC24215	2021-08-23	90.1	L452R, T478K, D614G, P681R, D950N, S974S,
1069	COVC24217	2021-08-23	93.4	T19R, L452R, T478K, Q613H, D614G, P681R, D950N...
936	KEMRI_02_91932	2021-10-02	98.8	G257S, D614G, P681H
896	KEMRI_01_89581	2021-12-01	96.5	H69_V70del, D138H, Y145del, N501Y, A570D, D614...

In [15]:
df_fort = df_data_dt.resample(rule='4W', closed='right', on='DT_SAM_COLL')

In [16]:
df_spike_trend = pd.DataFrame()
count = 1
observed_mut = []
for i,df in df_fort:
#     print(i, df)
    spike_mut_list = []
    for row in df.itertuples():
        if isinstance(row.S, str):
            mut_list = row.S.rstrip(', ').split(', ')
        else:
            pass#mut_list = []; mut_list.append(pd.NA)
        spike_mut_list.extend(mut_list)
    mut_count = Counter(spike_mut_list)
    df_new = pd.DataFrame(mut_count, index=[i])
    new_col = set(df_new.columns)
    output = sorted(list(new_col)); observed_mut.append(output)
    prev_col = set(df_spike_trend.columns)
    new_m_prev_col = new_col - prev_col
    new_u_prev_col = new_col & prev_col
    prev_m_new_prev_col = prev_col - new_u_prev_col
    if new_m_prev_col == new_col:
        if count == 1:
            output = sorted(list(new_col))#; observed_mut.append(output)
            print(f'Initially detected mutation(s) as of {i.strftime("%Y-%m-%d")} are:\n  {output}\nThe counts:\n  {dict([(k, mut_count[k]) for k in output])}')
        else:
            output = sorted(list(new_col))#; observed_mut.append(output)
            print(f'No new mutation(s) were detected as of {i.strftime("%Y-%m-%d")}')
    elif len(new_m_prev_col) > 0:
        output = sorted(list(new_m_prev_col))#; observed_mut.append(output)
        print(f'The following new mutation(s) had emerged as of {i.strftime("%Y-%m-%d")}:\n  {output}\nThe counts:\n  {dict([(k, mut_count[k]) for k in output])}')
    elif len(new_m_prev_col) == 0:
        output = sorted(list(prev_col))#; observed_mut.append(output)
        print(f'No new mutation(s) were detected as of {i.strftime("%Y-%m-%d")}')

    count += 1
    df_spike_trend = df_spike_trend.append(df_new)


Initially detected mutation(s) as of 2020-07-12 are:
  ['A263S', 'D614G']
The counts:
  {'A263S': 1, 'D614G': 5}
The following new mutation(s) had emerged as of 2020-08-09:
  ['A845S', 'F515F', 'G261D', 'L18F', 'N928N', 'R21I', 'S1252F', 'T618T', 'Y28Y']
The counts:
  {'A845S': 1, 'F515F': 3, 'G261D': 1, 'L18F': 1, 'N928N': 1, 'R21I': 1, 'S1252F': 4, 'T618T': 1, 'Y28Y': 1}
The following new mutation(s) had emerged as of 2020-09-06:
  ['A846S', 'N394I', 'P728P', 'S691S']
The counts:
  {'A846S': 1, 'N394I': 1, 'P728P': 4, 'S691S': 1}
The following new mutation(s) had emerged as of 2020-10-04:
  ['A27S', 'D80Y', 'E484K', 'G1085E', 'G832G', 'P330L', 'P561S', 'Q836R', 'R683R', 'S375S', 'T1273I', 'T638T', 'T73I', 'T941A', 'V1094V', 'V1228L', 'Y421Y']
The counts:
  {'A27S': 3, 'D80Y': 4, 'E484K': 2, 'G1085E': 1, 'G832G': 1, 'P330L': 1, 'P561S': 1, 'Q836R': 2, 'R683R': 4, 'S375S': 4, 'T1273I': 1, 'T638T': 1, 'T73I': 3, 'T941A': 1, 'V1094V': 4, 'V1228L': 1, 'Y421Y': 1}
The following new mutatio

### **Regions of interest mutation on spike**
 - 319-541 - RBD
 - 613-705 - S1/S2 junction

In [17]:
window_size = 3
interest_muts = []
for i in range(len(observed_mut) - window_size):
    period_slice = []; period_slice.extend(observed_mut[i:i+window_size])
#     print(len(period_slice))
    counter = Counter()
    for period in period_slice: 
#         print(len(period))
        cnt = Counter(period)
        counter += cnt
#         print(counter)
    for k, v in counter.items(): 
#         print(k,v)
        if v == len(period_slice):
#             if k not in interest_muts:
            interest_muts.append(k)
#             else: pass
        else: pass
print(f'The following mutations consistently appeared across {window_size} periods (are they emerging variants?):\n{dict(Counter(interest_muts))}')
#     print(period_slice)

moc_dict = dict(Counter(interest_muts))
moc_set = set(list(range(319,542)) + list(range(613,706)))
moc_dict_keys = set(moc_dict.keys())

moc_p = re.compile(r'[3-7][0-9]{2,2}')
moc_mut = set(map(lambda x: int(x), moc_p.findall(str(moc_dict_keys))))

moc = moc_set & moc_mut
if len(moc) > 0:
    p = re.compile('|'.join([str(i) for i in moc]))
    moc_fin = {k: v for k,v in moc_dict.items() if p.findall(k)}
    print(f'\n\nThe following consistent mutations occur in either the RBD (319-541) or the S1-S2 (613-705) spike regions:\n {moc_fin}')

The following mutations consistently appeared across 3 periods (are they emerging variants?):
{'D614G': 15, 'P728P': 2, 'D80Y': 1, 'G1085E': 1, 'Q836R': 1, 'R683R': 1, 'S375S': 1, 'T73I': 1, 'V1228L': 1, 'L1200L': 1, 'Q1201K': 1, 'A570D': 7, 'D1118H': 7, 'D138H': 6, 'D950N': 8, 'H69_V70del': 7, 'N501Y': 7, 'P681H': 7, 'S982A': 7, 'T19R': 8, 'T478K': 8, 'T716I': 7, 'Y145del': 7, 'A67V': 1, 'A701V': 1, 'D215G': 1, 'D80A': 1, 'E484K': 1, 'F1062F': 1, 'F490S': 5, 'F888L': 1, 'K417N': 1, 'L18F': 1, 'L242_L244del': 1, 'Q677H': 2, 'S940S': 1, 'G1251V': 1, 'G261D': 3, 'L452R': 6, 'P681R': 6, 'S1252C': 1, 'E156_R158delinsG': 3, 'F32F': 3, 'C361*': 1, 'C361S': 1, 'D253G': 1, 'G446V': 2, 'N61fs': 1, 'Q613H': 2, 'V1264L': 2, 'D1163D': 1, 'D215H': 1, 'E661D': 1}


The following consistent mutations occur in either the RBD (319-541) or the S1-S2 (613-705) spike regions:
 {'D614G': 15, 'R683R': 1, 'S375S': 1, 'N501Y': 7, 'P681H': 7, 'T478K': 8, 'A701V': 1, 'E484K': 1, 'F490S': 5, 'K417N': 1, 'Q677H':

In [18]:
df_spike_trend.sum().sort_values(ascending=False)[:20]

D614G               1245.0
P681R                647.0
D950N                617.0
L452R                607.0
T478K                605.0
T19R                 594.0
E156_R158delinsG     246.0
P681H                237.0
H69_V70del           230.0
Y145del              230.0
D1118H               217.0
A570D                216.0
S982A                213.0
T716I                211.0
N501Y                197.0
Q613H                 54.0
R683R                 41.0
D138H                 39.0
G261D                 33.0
S375S                 27.0
dtype: float64